In [16]:
import os
import pandas as pd
import json
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
from node2vec_utils import read_node_vecs, load_json_dict, invert_dict
from suspicion_tools import cosine_sim, cycle_suspicion_desc, cycle_suspicion_score, cycle_suspicion_for_agg
%matplotlib inline  
%load_ext autoreload
%autoreload 2
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
transactions = pd.read_csv('../../data/transactions.small.csv')
transactions['source'] = transactions['source'].astype(str)
transactions['target'] = transactions['target'].astype(str)
#transactions['date'] = pd.to_datetime(transactions.date)
#transactions['time'] = pd.to_datetime(transactions['time'])
transactions['amount'] = transactions.amount.astype(float)

In [18]:
transactions

,id,source,target,date,time,amount,currency
0,0a6dfc9d-c66e-4546-a2d7-3e910e6478e3,a9c9e8db-c796-4d87-bdba-bbc9bf683d40,98fdf8fb-ec37-42ed-a890-d862ef93e7bb,05/24/2017,00:47:36,457.952172,ISK
1,59328a0d-92bb-43a5-8925-5db6a693dde9,c81d20dd-8b65-43d1-bb07-6988fffa299f,d84e8a14-d531-497c-87cb-ee985099e9f9,12/26/2016,03:57:40,940.451548,IRR
2,f0326cb7-9c59-49d0-9944-781bbea9546b,124e9c28-03a7-4bf5-81e5-13c21513ad51,9e4c7d85-e510-4be8-9b6c-3442e2c9dadf,01/27/2015,02:56:57,371.643714,None
3,3a667343-bc3b-4038-86bb-66358b38db36,9e4c7d85-e510-4be8-9b6c-3442e2c9dadf,19ce1dd2-f9e8-4a59-9261-7bbd4dba0398,01/27/2015,12:56:29,372.561046,None
4,732d967d-647e-410e-b997-41180b4d51c3,779bbd4a-bb73-410e-a308-1da3138c1998,11c9de39-9ce8-44ab-9080-850f07b32a8e,05/27/2016,13:58:13,2260.001111,None
5,d93d064c-e823-425b-b174-808e0c4f18e7,81d2b670-3a26-42fd-9450-dc269a762736,da7c0dc7-3b58-470a-87ac-f9b82cf0aaff,03/26/2017,15:46:25,1749.687037,KGS
6,b94164eb-9fd1-4236-871f-d77181d26016,dbb47ff3-c1d4-456a-a095-fcba688d6d52,c942c40b-7110-4750-b92d-15ba13a04e57,08/30/2015,07:59:58,428.690525,None
7,9751c78f-fd5c-4786-958a-3a30f987e4f2,8349868b-8c95-4ae0-be76-28ddfbe9ed4a,71b46d67-b1c6-4a3e-9ce1-bb48452f1bda,05/24/2017,21:47:18,343.572126,None
8,dca6b9ee-577d-4fec-8e94-89dace0bf846,f1a3f6e2-a427-4b87-b4a0-20f3c9760e75,b3389f28-6eb1-4bc5-a271-c629b3b3d276,05/28/2015,10:59:28,945.974452,None
9,668d0260-88a9-4496-a6ac-92fc251590c7,311be468-c6ad-4ad4-8c59-c87ac6be6047,4a3414d5-6f1a-4861-a1fc-fb1729cb5fef,04/27/2015,15:17:37,197.480239,None


In [19]:
clients = pd.read_csv('../../data/clients.small.csv')
companies = pd.read_csv('../../data/companies.small.csv')

In [20]:
loopsjson = pd.read_json('loops.json', lines=True)

In [76]:
#Generate jsons for suspects.
for i in range(loopsjson.shape[0]):
    all_ids = []
    all_link_ids = []
    current_suspect = loopsjson.loc[i,0]['from']['id']
    f = open('../../json/'+current_suspect+'.json', mode='w')
    current_dict = {}
    current_nodes = []
    current_links = []
    
    for j in range(loopsjson.shape[1]):
        if loopsjson.loc[i,j] is not None:
            #Adding the from and to nodes
            curr_node = {}
            curr_node['id'] = loopsjson.loc[i,j]['from']['id']
            #print(curr_node)
            if (curr_node['id'] not in all_ids):
                if j==0:
                    curr_node['tag'] = 'suspect'
                else:
                    curr_node['tag'] = 'accomplice'
                all_ids.append(curr_node['id'])
                current_nodes.append(curr_node)
            if (loopsjson.loc[i,j]['to']['id'] not in all_ids):
                all_ids.append(loopsjson.loc[i,j]['to']['id'])
                other_node = {}
                other_node['id'] = loopsjson.loc[i,j]['to']['id']
                other_node['tag'] = 'accomplice'
                current_nodes.append(other_node)
            
            current_link_id = loopsjson.loc[i,j]['id']
            current_link = transactions[transactions.id == current_link_id].to_dict(orient='records')
            current_link = current_link[0]
            if (current_link_id not in all_link_ids):
                current_link['tag'] = 'accomplice'
                current_links.append(current_link)
                all_link_ids.append(current_link_id)
                if (current_link['target'] not in all_ids):
                    node = {}
                    node['tag'] = 'accomplice'
                    node['id'] = current_link['target']
                    current_nodes.append(node)
                    all_ids.append(node['id'])
                if (current_link['source'] not in all_ids):
                    node = {}
                    node['tag'] = 'accomplice'
                    node['id'] = current_link['source']
                    current_nodes.append(node)
                    all_ids.append(node['id'])
            if (j == 0):
                root_node_id = loopsjson.loc[i,0]['from']['id']
                outgoing_links = transactions[transactions.source == root_node_id]
                if (outgoing_links.shape[0] >= 5):
                    sampled = outgoing_links.sample(n=5).drop_duplicates(subset=['id']).to_dict(orient='records')
                else:
                    sampled = outgoing_links.drop_duplicates(subset=['id']).to_dict(orient='records')
                for current_link in sampled:
                    if (current_link['id'] not in all_link_ids):
                        current_link['tag'] = 'usual'
                        current_links.append(current_link)
                        all_link_ids.append(current_link['id'])
                        if (current_link['target'] not in all_ids):
                            node = {}
                            node['tag'] = 'usual'
                            node['id'] = current_link['target']
                            #print(node)
                            current_nodes.append(node)
                            all_ids.append(node['id'])
                        if (current_link['source'] not in all_ids):
                            node = {}
                            node['tag'] = 'usual'
                            node['id'] = current_link['source']
                            #print(node)
                            current_nodes.append(node)
                            all_ids.append(node['id'])
            
    current_dict['nodes'] = current_nodes
    current_dict['links'] = current_links
    json.dump(current_dict, f)
    f.close()
    

In [ ]:
#Generate some example jsons for non-suspects.